In [1]:
# Importa as bibliotecas
import pandas as pd
import numpy as np

In [2]:
# Carrega a planilha com os dados
df = pd.read_excel("assets/Planilha novembro 2025.xlsx")

# Exibe as 5 primeiras linha do df
df.head()

,Empresa,Código,Atuação,Quantidade total de ações,Valor de mercado,Lucro líquido estimado 2025,P/L projetado,P/L médio (últ. 10 anos),Desvio do P/L da sua média,CAGR lucros (últ. 5 anos),...,Lucro por ação estimado,Payout esperado,Dividendo por ação bruto projetado,Dividend Yield bruto estimado,Cotação atual,Preço Teto,Margem de segurança,Frequência nos anúncios,Meses que costumam anunciar dividendos,Última atualização
0,Axia Energia,AXIA3,Energia,"2,247,634,148.00",NaN,"R$ 8,000,000,000.00",NaN,9.3,NaN,0.8%,...,R$ 3.56,100.00%,R$ 3.56,NaN,NaN,R$ 42.00,NaN,Semestral,Abril e dezembro,01/09/2025
1,SLC Agrícola,SLCE3,Agronegócio,"441,205,368.00","R$ 6,318,060,870.00","R$ 840,000,000.00",7.5,10.8,-30.4%,8.4%,...,R$ 1.90,50.00%,R$ 0.95,6.6%,R$ 14.32,R$ 22.33,36%,Semestral,abril e dezembro,08/11/2025
2,Petroreconcâvo,RECV3,"Oléo, gás e biocomb.","292,973,655.00","R$ 3,184,623,630.00","R$ 650,000,000.00",4.9,9.3,-47.3%,31.7%,...,R$ 2.22,40.00%,R$ 0.89,8.2%,R$ 10.87,R$ 14.00,22%,Semestral,maio e dez,07/11/2025
3,BB Seguridade,BBSE3,Seguros,"1,941,214,909.00","R$ 69,417,845,146.00","R$ 8,800,000,000.00",7.9,12.1,-34.8%,18.3%,...,R$ 4.53,88.00%,R$ 3.99,11.2%,R$ 35.76,R$ 45.33,21%,Semestral,fevereiro e agosto,08/11/2025
4,Wiz,WIZC3,Seguros,"159,907,000.00","R$ 1,466,347,190.00","R$ 210,000,000.00",7.0,8.6,-18.8%,11.1%,...,R$ 1.31,50.00%,R$ 0.66,7.2%,R$ 9.17,R$ 10.51,13%,Anual,abril,08/11/2025


In [3]:
# Exibe as informações básicas do df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 21 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Empresa                                 81 non-null     object 
 1   Código                                  81 non-null     object 
 2   Atuação                                 81 non-null     object 
 3   Quantidade total de ações               81 non-null     object 
 4   Valor de mercado                        80 non-null     object 
 5   Lucro líquido estimado 2025             81 non-null     object 
 6   P/L projetado                           80 non-null     float64
 7   P/L médio (últ. 10 anos)                81 non-null     float64
 8   Desvio do P/L da sua média              80 non-null     object 
 9   CAGR lucros (últ. 5 anos)               81 non-null     object 
 10  Dívida líquida/EBITDA                   81 non-null     float64


In [4]:
# Cria uma cópia do DataFrame original para evitar alterações nos dados brutos
df_clean = df.copy()

# Função responsável por remover símbolos percentuais,
# padronizar o separador decimal e converter os valores para float
def limpar_percentual(col):
    return (
        col.str.replace("%", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
    )


# Converte a coluna de Dividend Yield bruto estimado para formato numérico
df_clean["Dividend Yield bruto estimado"] = limpar_percentual(
    df_clean["Dividend Yield bruto estimado"]
)

# Converte a coluna de Margem de segurança para formato numérico
df_clean["Margem de segurança"] = limpar_percentual(df_clean["Margem de segurança"])

# Garante que a coluna P/L projetado esteja em formato numérico
df_clean["P/L projetado"] = df_clean["P/L projetado"].astype(float)

# Remove empresas com dados ausentes nas métricas essenciais para o cálculo do score
df_clean = df_clean.dropna(
    subset=["Dividend Yield bruto estimado", "Margem de segurança", "P/L projetado"]
)

In [5]:
# Normaliza o Dividend Yield bruto estimado,
# atribuindo valores maiores às empresas com maior retorno em dividendos
df_clean["DY_norm"] = (
    df_clean["Dividend Yield bruto estimado"]
    / df_clean["Dividend Yield bruto estimado"].max()
)

# Normaliza a Margem de segurança,
# priorizando empresas com maior desconto em relação ao preço justo
df_clean["MS_norm"] = (
    df_clean["Margem de segurança"] / df_clean["Margem de segurança"].max()
)

# Normaliza o P/L projetado de forma inversa,
# garantindo que empresas com menor P/L recebam maior pontuação
df_clean["PL_norm"] = df_clean["P/L projetado"].min() / df_clean["P/L projetado"]

In [6]:
# Calcula o score final de cada empresa a partir da soma dos indicadores normalizados,
# representando o equilíbrio entre retorno em dividendos, margem de segurança e valuation
df_clean["Score"] = (
    df_clean["DY_norm"] + df_clean["MS_norm"] + df_clean["PL_norm"]
).round(2)

In [7]:
# Ordena as empresas pelo score final em ordem decrescente,
# seleciona as 10 melhores classificadas
# e mantém apenas as colunas relevantes para análise e apresentação
df_top10 = (
    df_clean.sort_values("Score", ascending=False)
    .head(10)
    .loc[
        :,
        [
            "Empresa",
            "Código",
            "Atuação",
            "Dividend Yield bruto estimado",
            "Margem de segurança",
            "P/L projetado",
            "Score",
        ],
    ]
    .reset_index(drop=True)
)

# Exibe o DataFrame com as Top 10 ações segundo os critérios definidos
df_top10

,Empresa,Código,Atuação,Dividend Yield bruto estimado,Margem de segurança,P/L projetado,Score
0,Petrobras,PETR4,"Oléo, gás e biocomb.",11.8,22.0,3.8,2.30
1,Metalúrgica Gerdau,GOAU3,Siderurgia e Metalurgia,10.8,26.0,6.5,2.04
2,SLC Agrícola,SLCE3,Agronegócio,6.6,36.0,7.5,1.91
3,Marcopolo,POMO3,Industrial,9.5,21.0,5.3,1.88
4,BB Seguridade,BBSE3,Seguros,11.2,21.0,7.9,1.86
5,Petroreconcâvo,RECV3,"Oléo, gás e biocomb.",8.2,22.0,4.9,1.84
6,Klabin,KLBN11,Papel e Celulose,5.8,8.0,2.6,1.71
7,Klabin,KLBN4,Papel e Celulose,5.8,8.0,2.6,1.71
8,Grendene,GRND3,Calçados,9.5,16.0,5.8,1.70
9,Marcopolo,POMO4,Industrial,9.0,17.0,5.6,1.70
